In [1]:
import pandas as pd
import numpy as np
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import random
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import os
from sklearn.preprocessing import StandardScaler
from numpy.random import seed

In [3]:
# data load
# path 변수를 적절히 변경
x_train_path = os.path.join('data/train_features.csv')
y_train_path = os.path.join('data/train_labels.csv')
x_test_path = os.path.join('data/test_features.csv')
sub_path = os.path.join('data/sample_submission.csv')

train = pd.read_csv(x_train_path)
train_label = pd.read_csv(y_train_path)
test = pd.read_csv(x_test_path)
sub = pd.read_csv(sub_path)

In [4]:
train['acc_Energy']=(train['acc_x']**2+train['acc_y']**2+train['acc_z']**2)**(1/3)
test['acc_Energy']=(test['acc_x']**2+test['acc_y']**2+test['acc_z']**2)**(1/3)

train['gy_Energy']=(train['gy_x']**2+train['gy_y']**2+train['gy_z']**2)**(1/3)
test['gy_Energy']=(test['gy_x']**2+test['gy_y']**2+test['gy_z']**2)**(1/3)

train['gy_acc_Energy']=((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/3)
test['gy_acc_Energy']=((test['gy_x']-test['acc_x'])**2+(test['gy_y']-test['acc_y'])**2+(test['gy_z']-test['acc_z'])**2)**(1/3)

In [5]:
dt=0.02 
def jerk_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [6]:
train_dt=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    train_dt.append(temp)

100%|██████████████████████████████████████| 3125/3125 [00:27<00:00, 115.08it/s]


In [7]:
test_dt=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    test_dt.append(temp)

100%|████████████████████████████████████████| 782/782 [00:06<00:00, 128.62it/s]


In [8]:
from scipy import fftpack
from numpy.fft import *

def fourier_transform_one_signal(t_signal):
    complex_f_signal= fftpack.fft(t_signal)
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal

In [9]:
train=pd.concat(train_dt)

In [10]:
fft=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for i in train.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft.append(temp)
train=pd.concat(fft)

100%|██████████████████████████████████████| 3125/3125 [00:05<00:00, 529.03it/s]


In [11]:
test=pd.concat(test_dt)

In [12]:
fft_t=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for i in test.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft_t.append(temp)
test=pd.concat(fft_t)

100%|████████████████████████████████████████| 782/782 [00:00<00:00, 880.43it/s]


In [13]:
col=train.columns
train_s=train.copy()
test_s=test.copy()

In [14]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

scaler = StandardScaler()

train_s.iloc[:,2:]= scaler.fit_transform(train_s.iloc[:,2:])
train_sc = pd.DataFrame(data = train_s,columns =col)

test_s.iloc[:,2:]= scaler.transform(test_s.iloc[:,2:])
test_sc = pd.DataFrame(data = test_s,columns =col)

In [15]:
# make dataset
# x를 시계열 데이터로 변경 시켜주는 함수
def make_dataset(data):
  ids = data['id'].unique()
  id_data = data.groupby('id')
  series_data = []

  for i in ids:
    df = id_data.get_group(i)
    df = df.drop(['id', 'time'], axis=1)
    series_data.append(df.to_numpy())

  series_data = np.array(series_data)
  return series_data

In [16]:
series_train = make_dataset(train_sc)
series_test = make_dataset(test_sc)

In [17]:
series_train.shape, series_test.shape

((3125, 600, 18), (782, 600, 18))

In [18]:
# label one-hot 인코딩
labels = train_label['label'].to_numpy()

In [19]:
import tensorflow as tf

2021-10-03 22:55:53.292328: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [20]:
# dataset과 validation set을 만들어 주는 함수
# validation set은 shuffle 적용 x
def make_train(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 64
  train_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  train_dataset = train_dataset.batch(BATCH_SIZE)
  train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

def make_val(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 64
  val_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  val_dataset = val_dataset.batch(BATCH_SIZE)
  val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

In [21]:
from tensorflow import keras
from tensorflow.keras import backend as K

In [33]:
# 가중치 적용 loss
# https://openaccess.thecvf.com/content_CVPR_2019/papers/Cui_Class-Balanced_Loss_Based_on_Effective_Number_of_Samples_CVPR_2019_paper.pdf
# 위 논문 참고
def weighted_loss(labels, y_pred, beta=0.9):
        no_samples = [12, 21, 20, 23, 35, 25, 24, 26, 97, 37, 20, 23, 12,
                     12, 25, 25, 22, 27, 47, 20, 26, 27, 19, 20, 35, 24,
                     1518, 34, 55, 20, 35, 20, 18, 20, 22, 30, 28, 35, 20,
                     20, 34, 20, 20, 35, 21, 22, 20, 26, 25, 30, 37, 24,
                     12, 13, 23, 37, 36, 20, 20, 23, 48]

        weight = tf.divide(1-beta, 1-tf.pow(beta, no_samples))
        #y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        loss = labels * K.log(y_pred) * weight
        loss = -K.sum(loss, -1)
        return loss

def focal(labels, y_pred, gamma=2):
    #y_pred = tf.nn.softmax(logits, dim=-1)  # [batch_size, num_classes]
    #labels = tf.squeeze(labels)  # label example: [0,1,2,3]
    #labels = tf.to_float(tf.one_hot(labels, depth=y_pred.shape[1]))

    loss = -labels * ((1 - y_pred) ** gamma) * K.log(y_pred)
    loss = tf.reduce_mean(tf.reduce_sum(loss, axis=1))
    return loss


# 모델을 만들어 주는 함수
def base():
    seed(2021)
    tf.random.set_seed(2021)
    model = keras.models.Sequential([
            keras.layers.Conv1D(filters=128, kernel_size=9, padding='same', input_shape=[600, 18]),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.3),
            keras.layers.Conv1D(filters=256, kernel_size=6, padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.4),
            keras.layers.Conv1D(filters=128, kernel_size=3,padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.5),
            keras.layers.GlobalAveragePooling1D(),
            keras.layers.Dense(61, activation='softmax')
    ])
    model.compile(optimizer='adam',
                loss='categorical_crossentropy', 
                metrics=['accuracy'])
    return model

In [26]:
# checkpoint path
# 중간중간 모델의 weight를 저장할 경로 설정
ckpt_name = 'fold_cnn_weighted_ckpt.hdf5'
checkpoint_dir_path = os.path.join('checkpoint')
checkpoint_path = os.path.join(checkpoint_dir_path, ckpt_name)

# callback 함수 목록
callbacks_list = [
    # 매 epoch 마다 val_loss를 체크하여 가장 낮은 상태의 weight를 저장
    tf.keras.callbacks.ModelCheckpoint(
        filepath = checkpoint_path,
        monitor='val_loss',
        mode='min',
        save_weights_only=True,
        save_best_only=True
    ),
    # 8번 동안 val_loss의 향상이 없으면 훈련 종료
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode='min',
        verbose=1, 
        patience=8
    ),
    tf.keras.callbacks.ReduceLROnPlateau(patience = 4,verbose = 1,factor = 0.5) 
]

In [27]:
from sklearn.model_selection import StratifiedKFold

In [28]:
no_samples = [12, 21, 20, 23, 35, 25, 24, 26, 97, 37, 20, 23, 12,
                     12, 25, 25, 22, 27, 47, 20, 26, 27, 19, 20, 35, 24,
                     1518, 34, 55, 20, 35, 20, 18, 20, 22, 30, 28, 35, 20,
                     20, 34, 20, 20, 35, 21, 22, 20, 26, 25, 30, 37, 24,
                     12, 13, 23, 37, 36, 20, 20, 23, 48]

beta = 0.9
weight = (1.0-beta) / (1.0-np.power(beta, no_samples))

In [31]:
weight = {i: w for i, w in enumerate(weight)}

In [34]:
# k-fold 적용
# 데이터 수가 적어 10개의 fold 사용
k = 10 
split = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
results = []    # 매 fold 훈련 후 loss 저장
models = []     # 매 fold 훈련 후 해당 모델 저장
accs = []
for i, (train, val) in enumerate(split.split(series_train, labels)):
    print('Fold', i)
    print('#'*20)

    train_dataset = make_train(series_train[train], labels[train])
    val_dataset = make_val(series_train[val], labels[val])

    model = base()

    model.fit(train_dataset, validation_data=val_dataset, callbacks=callbacks_list, 
              class_weight=weight, epochs=100)

    # 모델 복원
    model.load_weights(checkpoint_path)
    result = model.evaluate(val_dataset)[0]
    acc = model.evaluate(val_dataset)[1]
    
    results.append(result)
    accs.append(acc)
    models.append(model)

    del model, train_dataset, val_dataset

Fold 0
####################
Epoch 1/100


2021-10-03 22:59:29.001679: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-10-03 22:59:29.126328: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7


44/44 [==============================] - 1s 31ms/step - loss: 0.3009 - accuracy: 0.4566 - val_loss: 3.3506 - val_accuracy: 0.2852
Epoch 2/100
44/44 [==============================] - 1s 29ms/step - loss: 0.2108 - accuracy: 0.5487 - val_loss: 1.8709 - val_accuracy: 0.5423
Epoch 3/100
44/44 [==============================] - 1s 28ms/step - loss: 0.1862 - accuracy: 0.5782 - val_loss: 1.8885 - val_accuracy: 0.5562
Epoch 4/100
44/44 [==============================] - 1s 29ms/step - loss: 0.1658 - accuracy: 0.6177 - val_loss: 1.7360 - val_accuracy: 0.5669
Epoch 5/100
44/44 [==============================] - 1s 29ms/step - loss: 0.1490 - accuracy: 0.6454 - val_loss: 1.5436 - val_accuracy: 0.5992
Epoch 6/100
44/44 [==============================] - 1s 29ms/step - loss: 0.1344 - accuracy: 0.6746 - val_loss: 1.3550 - val_accuracy: 0.6334
Epoch 7/100
44/44 [==============================] - 1s 27ms/step - loss: 0.1214 - accuracy: 0.7023 - val_loss: 1.2285 - val_accuracy: 0.6682
Epoch 8/100
44/44 

44/44 [==============================] - 1s 29ms/step - loss: 0.0099 - accuracy: 0.9851 - val_loss: 0.1566 - val_accuracy: 0.9619
Epoch 58/100
44/44 [==============================] - 1s 29ms/step - loss: 0.0093 - accuracy: 0.9868 - val_loss: 0.1521 - val_accuracy: 0.9630
Epoch 59/100
44/44 [==============================] - 1s 30ms/step - loss: 0.0090 - accuracy: 0.9872 - val_loss: 0.1448 - val_accuracy: 0.9637
Epoch 60/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0091 - accuracy: 0.9879 - val_loss: 0.1642 - val_accuracy: 0.9545
Epoch 61/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0084 - accuracy: 0.9904 - val_loss: 0.1568 - val_accuracy: 0.9520
Epoch 62/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0081 - accuracy: 0.9900 - val_loss: 0.1478 - val_accuracy: 0.9602
Epoch 63/100
44/44 [==============================] - 1s 30ms/step - loss: 0.0076 - accuracy: 0.9915 - val_loss: 0.1394 - val_accuracy: 0.9641
Epoch 64/100

44/44 [==============================] - 1s 26ms/step - loss: 0.0608 - accuracy: 0.8432 - val_loss: 0.5972 - val_accuracy: 0.8442
Epoch 18/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0574 - accuracy: 0.8492 - val_loss: 0.5643 - val_accuracy: 0.8496
Epoch 19/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0544 - accuracy: 0.8631 - val_loss: 0.5500 - val_accuracy: 0.8506
Epoch 20/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0518 - accuracy: 0.8674 - val_loss: 0.5140 - val_accuracy: 0.8602
Epoch 21/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0498 - accuracy: 0.8706 - val_loss: 0.5037 - val_accuracy: 0.8602
Epoch 22/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0471 - accuracy: 0.8770 - val_loss: 0.4874 - val_accuracy: 0.8695
Epoch 23/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0453 - accuracy: 0.8830 - val_loss: 0.4923 - val_accuracy: 0.8634
Epoch 24/100

...

44/44 [==============================] - 1s 27ms/step - loss: 0.0234 - accuracy: 0.9499 - val_loss: 0.2583 - val_accuracy: 0.9307
Epoch 41/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0226 - accuracy: 0.9531 - val_loss: 0.2517 - val_accuracy: 0.9325
Epoch 42/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0217 - accuracy: 0.9566 - val_loss: 0.2447 - val_accuracy: 0.9364
Epoch 43/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0211 - accuracy: 0.9584 - val_loss: 0.2406 - val_accuracy: 0.9371
Epoch 44/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0207 - accuracy: 0.9609 - val_loss: 0.2310 - val_accuracy: 0.9417
Epoch 45/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0200 - accuracy: 0.9577 - val_loss: 0.2331 - val_accuracy: 0.9428
Epoch 46/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0194 - accuracy: 0.9623 - val_loss: 0.2342 - val_accuracy: 0.9435
Epoch 47/100

Epoch 94/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0087 - accuracy: 0.9918 - val_loss: 0.1402 - val_accuracy: 0.9691
Epoch 95/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0087 - accuracy: 0.9908 - val_loss: 0.1380 - val_accuracy: 0.9691
Epoch 96/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0086 - accuracy: 0.9911 - val_loss: 0.1370 - val_accuracy: 0.9680
Epoch 97/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0084 - accuracy: 0.9932 - val_loss: 0.1370 - val_accuracy: 0.9694
Epoch 98/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0086 - accuracy: 0.9922 - val_loss: 0.1387 - val_accuracy: 0.9691
Epoch 99/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0086 - accuracy: 0.9925 - val_loss: 0.1371 - val_accuracy: 0.9687
Epoch 100/100
44/44 [==============================] - 0s 8ms/step - loss: 0.1649 - accuracy: 0.9588
Fold 8
####################
Epoch 1/100
4

Epoch 49/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0149 - accuracy: 0.9730 - val_loss: 0.2410 - val_accuracy: 0.9289
Epoch 50/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0149 - accuracy: 0.9723 - val_loss: 0.1963 - val_accuracy: 0.9563
Epoch 51/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0145 - accuracy: 0.9726 - val_loss: 0.2100 - val_accuracy: 0.9520
Epoch 52/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0136 - accuracy: 0.9762 - val_loss: 0.2369 - val_accuracy: 0.9271
Epoch 53/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0135 - accuracy: 0.9758 - val_loss: 0.1865 - val_accuracy: 0.9549
Epoch 54/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0129 - accuracy: 0.9769 - val_loss: 0.1909 - val_accuracy: 0.9527
Epoch 55/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0130 - accuracy: 0.9762 - val_loss: 0.1986 - val_accuracy: 0.9477

44/44 [==============================] - 1s 29ms/step - loss: 0.3001 - accuracy: 0.4589 - val_loss: 3.2759 - val_accuracy: 0.3075
Epoch 2/100
44/44 [==============================] - 1s 26ms/step - loss: 0.2103 - accuracy: 0.5489 - val_loss: 1.8633 - val_accuracy: 0.5446
Epoch 3/100
44/44 [==============================] - 1s 27ms/step - loss: 0.1856 - accuracy: 0.5848 - val_loss: 1.8534 - val_accuracy: 0.5542
Epoch 4/100
44/44 [==============================] - 1s 27ms/step - loss: 0.1650 - accuracy: 0.6196 - val_loss: 1.7677 - val_accuracy: 0.5624
Epoch 5/100
44/44 [==============================] - 1s 27ms/step - loss: 0.1486 - accuracy: 0.6470 - val_loss: 1.5562 - val_accuracy: 0.5997
Epoch 6/100
44/44 [==============================] - 1s 28ms/step - loss: 0.1345 - accuracy: 0.6665 - val_loss: 1.3608 - val_accuracy: 0.6324
Epoch 7/100
44/44 [==============================] - 1s 28ms/step - loss: 0.1218 - accuracy: 0.6993 - val_loss: 1.2125 - val_accuracy: 0.6665
Epoch 8/100
44/44 

44/44 [==============================] - 1s 28ms/step - loss: 0.0127 - accuracy: 0.9744 - val_loss: 0.1906 - val_accuracy: 0.9541
Epoch 58/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0105 - accuracy: 0.9851 - val_loss: 0.1710 - val_accuracy: 0.9605
Epoch 59/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0090 - accuracy: 0.9876 - val_loss: 0.1534 - val_accuracy: 0.9623
Epoch 60/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0084 - accuracy: 0.9904 - val_loss: 0.1576 - val_accuracy: 0.9623
Epoch 61/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0077 - accuracy: 0.9929 - val_loss: 0.1521 - val_accuracy: 0.9637
Epoch 62/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0071 - accuracy: 0.9943 - val_loss: 0.1474 - val_accuracy: 0.9645
Epoch 63/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0069 - accuracy: 0.9936 - val_loss: 0.1560 - val_accuracy: 0.9616
Epoch 64/100

In [35]:
results

[0.11166270822286606,
 0.17303675413131714,
 0.16791124641895294,
 0.10294482111930847,
 0.14986839890480042,
 0.09983431547880173,
 0.16388005018234253,
 0.16492170095443726,
 0.16470849514007568,
 0.1696777641773224]

In [36]:
np.mean(accs)

0.9627379775047302

In [37]:
# 결과 생성
pred_list = []    # 예측 결과를 담을 리스트
for model in models:
  pred = model.predict(series_test)
  pred_list.append(pred)

pred = np.mean(pred_list, axis=0)

In [38]:
# 제출물 생성
sub.iloc[:, 1:] = pred

save_path = os.path.join('sub/test_wt.csv')     # 저장 경로
sub.to_csv(save_path, index=False)